


<h1 style='font-size:36px; font-weight:bold;'>API-დან მონაცემების წამოღება, ბაზასთან დაკავშირება და ჩატვირთვა   </h1>

<br>




In [67]:
import requests
import os
import json
import csv
import mysql.connector

# API  configuration
API_KEY = " "
HEADERS = {"X-API-KEY": API_KEY}
BASE_URL = "https://api.opensea.io/api/v2/collections"
params = {"chain": "ethereum", "limit": 100} 

# configuration of database
DB_CONFIG = {
    "host": "localhost",
    "user": "root",
    "password": " ",
    "database": "opensea_db"
}

# File Paths
RAW_JSON_FILE = "opensea_data.json"
RAW_CSV_FILE = "opensea_data.csv"

# connecting to  MySQL
def connect_db():
    return mysql.connector.connect(**DB_CONFIG)

# creating table
def create_table():
    db = connect_db()
    cursor = db.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS collections (
            id INT AUTO_INCREMENT PRIMARY KEY,
            collection VARCHAR(255) UNIQUE,
            name VARCHAR(255),
            description TEXT,
            image_url TEXT,
            owner VARCHAR(255),
            twitter_username VARCHAR(255),
            contracts JSON
        )
    ''')
    db.commit()
    cursor.close()
    db.close()
    print("✅ Table created or already exists.")

# extracting data 
def extract_data():
    response = requests.get(BASE_URL, headers=HEADERS, params=params)
    if response.status_code == 200:
        raw_data = response.json()
        
        # Save raw JSON
        with open(RAW_JSON_FILE, "w") as f:
            json.dump(raw_data, f, indent=4)
        
        print(f"✅ Raw data saved to {RAW_JSON_FILE}")
        return raw_data.get("collections", [])
    else:
        print(f"❌ API Error: {response.status_code} - {response.text}")
        return []

# Transform Data
def transform_data(collections):
    transformed = []
    print(f"🔄 Transforming {len(collections)} collections...")

    for item in collections:
        # Data Cleaning
        collection_slug = item.get("slug", "").strip() or "unknown_collection"
        collection_name = item.get("name", "Unnamed Collection").strip()
        collection_description = item.get("description", "No description available").strip()
        image_url = item.get("image_url", None)
        owner = item.get("primary_owner", {}).get("user", {}).get("username", "Unknown Owner")
        twitter_username = item.get("twitter_username", None)

        # Data Mapping
        contracts = item.get("contracts", [])
        contracts_json = json.dumps(contracts) if contracts else "[]"

        transformed.append(
            (collection_slug, collection_name, collection_description, image_url, owner, twitter_username, contracts_json)
        )

    print(f"✅ Transformation complete. {len(transformed)} records processed.")
    return transformed


# Save Data to CSV (Data Lake)
def save_data_to_csv(transformed):
    with open(RAW_CSV_FILE, mode="w", newline="", encoding="utf-8") as f:  # Set encoding to utf-8
        writer = csv.writer(f)
        writer.writerow(["collection", "name", "description", "image_url", "owner", "twitter_username", "contracts"])
        writer.writerows(transformed)
    print(f"✅ Raw data saved to {RAW_CSV_FILE}")


# Load Data into Database
def load_data(transformed):
    db = connect_db()
    cursor = db.cursor()
    sql = '''
        INSERT INTO collections (collection, name, description, image_url, owner, twitter_username, contracts)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
        ON DUPLICATE KEY UPDATE 
        name = VALUES(name),
        description = VALUES(description),
        image_url = VALUES(image_url),
        owner = VALUES(owner),
        twitter_username = VALUES(twitter_username),
        contracts = VALUES(contracts)
    '''
    cursor.executemany(sql, transformed)
    db.commit()
    cursor.close()
    db.close()
    print(f"✅  {len(transformed)} records loaded into MySQL database.")

# Run ETL
if __name__ == "__main__":
    create_table()
    extracted_data = extract_data()
    if extracted_data:
        transformed_data = transform_data(extracted_data)
        # Save transformed data to CSV
        save_data_to_csv(transformed_data)
          # Load into MySQL
        load_data(transformed_data)
        print("🎉 ETL Pipeline Completed Successfully!")
    else:
        print(" No data extracted. ETL pipeline stopped.")


✅ Table created or already exists.
✅ Raw data saved to opensea_data.json
🔄 Transforming 100 collections...
✅ Transformation complete. 100 records processed.
✅ Raw data saved to opensea_data.csv
✅  100 records loaded into MySQL database.
🎉 ETL Pipeline Completed Successfully!





<h1 style='font-size:36px; font-weight:bold;'> არასტრუქტურირებული მონაცემები </h1>

<br>






In [61]:
print(transformed_data)

[('', 'naked flwer', '', '', 'Unknown Owner', '', '[{"address": "0x54acfbe5c26ab1f9204a731e703ba982b6a314d9", "chain": "ethereum"}]', 1), ('', 'SHOGUN', 'SHOGUN is a one-of-a-kind art collection by TSUKASA TAMAI, featuring imaginative samurai warriors drawn with pen and ink. These unique characters blend history and fantasy, embodying the spirit of the samurai with an eccentric twist. Each piece is a singular original—adopt a new general into your collection today.', 'https://i.seadn.io/s/raw/files/d191df7e6d4da0b2dda064c08b37007d.jpg?w=500&auto=format', 'Unknown Owner', '', '[{"address": "0x4bdb3334edd34a353b37ace4008725d622675932", "chain": "ethereum"}]', 1), ('', 'The Bengal Tiger', '', '', 'Unknown Owner', '', '[{"address": "0xd4284d44dda9b47baeb195fa81a2e10e3045b36c", "chain": "ethereum"}]', 1), ('', 'Red Ribbon', '', 'https://i.seadn.io/s/raw/files/e7e2af6a927a73b0fc3df52ceadcdcb8.jpg?w=500&auto=format', 'Unknown Owner', '', '[{"address": "0x9fae9d50888962aceee9a7ffe38ebb87bd6cff


<br>

# მონაცემების საერთო სურათის დანახვა

<br>



In [79]:

import pandas as pd

# Fetch data from 'collections' table
def fetch_data():
    db = connect_db()
    cursor = db.cursor(dictionary=True)
    cursor.execute("SELECT * FROM collections")
    result = cursor.fetchall()
    cursor.close()
    db.close()
    return result

# Display data in a DataFrame
data = fetch_data()


df = pd.DataFrame(data)


print(df)
data


      id          collection                         name  \
0      1                None                Right to Know   
1      2                None                      ZIPANGU   
2      3                None                   Hiphop DOG   
3      4                None                     NxTLvLAI   
4      5                None                     NxTLvLAI   
..   ...                 ...                          ...   
196  197                None         Radiant Elf Sentinel   
197  198                None                       Daniel   
198  199                None        My personal Severance   
199  200                None                 ROADLİFESİGN   
200  201  unknown_collection  Peaceful Persian Collection   

                                           description  \
0                                                        
1    Neo-Edo City in the far future. There, a world...   
2                                                        
3                                  

[{'id': 1,
  'collection': None,
  'name': 'Right to Know',
  'description': '',
  'image_url': 'https://i.seadn.io/s/raw/files/f280f03afbe3df11d8dced8b6c8ee25e.jpg?w=500&auto=format',
  'owner': 'Unknown Owner',
  'twitter_username': '',
  'contracts': '[{"chain": "ethereum", "address": "0x0588ca5c2b3dd96c9ce56b14df6a8620d6c6e891"}]'},
 {'id': 2,
  'collection': None,
  'name': 'ZIPANGU',
  'description': 'Neo-Edo City in the far future. There, a world where technology and tradition intertwine spread out. In this city where powerful corporations and rebel organizations seeking freedom wage a daily battle, defeat the corrupt corporations that control the city.',
  'image_url': 'https://i.seadn.io/s/raw/files/d0f51b5a56fdb3ae87b0a23921c8e11e.jpg?w=500&auto=format',
  'owner': 'Unknown Owner',
  'twitter_username': '',
  'contracts': '[{"chain": "ethereum", "address": "0x59cfac9bb25d7afbcecfd6f66ec5b00b20a488f5"}]'},
 {'id': 3,
  'collection': None,
  'name': 'Hiphop DOG',
  'description




<h1 style='font-size:36px; font-weight:bold;'>კოლექციის პირველი 20 ნიმუში </h1>

<br>




In [51]:
# Display the first 20 rows, for example:
df.iloc[:20]


,id,collection,name,description,image_url,owner,twitter_username,contracts
0,1,None,Right to Know,,https://i.seadn.io/s/raw/files/f280f03afbe3df1...,Unknown Owner,,"[{""chain"": ""ethereum"", ""address"": ""0x0588ca5c2..."
1,2,None,ZIPANGU,"Neo-Edo City in the far future. There, a world...",https://i.seadn.io/s/raw/files/d0f51b5a56fdb3a...,Unknown Owner,,"[{""chain"": ""ethereum"", ""address"": ""0x59cfac9bb..."
2,3,None,Hiphop DOG,,https://i.seadn.io/s/raw/files/c2e94dfea1277de...,Unknown Owner,,"[{""chain"": ""ethereum"", ""address"": ""0xe9ff61236..."
3,4,None,NxTLvLAI,,,Unknown Owner,,"[{""chain"": ""ethereum"", ""address"": ""0xaa358a530..."
4,5,None,NxTLvLAI,,,Unknown Owner,,"[{""chain"": ""ethereum"", ""address"": ""0xbeacbce37..."
5,6,None,flower shop,,,Unknown Owner,,"[{""chain"": ""ethereum"", ""address"": ""0x7b86deb8f..."
6,7,None,flower shop,,,Unknown Owner,,"[{""chain"": ""ethereum"", ""address"": ""0x7b5b22c40..."
7,8,None,Normal Saline,,https://i.seadn.io/s/raw/files/897f07974d4ac3d...,Unknown Owner,,"[{""chain"": ""ethereum"", ""address"": ""0x17a377498..."
8,9,None,Hua Hua,,https://i.seadn.io/s/raw/files/a3b55a44175fe47...,Unknown Owner,,"[{""chain"": ""ethereum"", ""address"": ""0xdee07e355..."
9,10,None,Hip Hop Dog,,https://i.seadn.io/s/raw/files/5aa68e40e1c1d03...,Unknown Owner,,"[{""chain"": ""ethereum"", ""address"": ""0x8f6fe7015..."
